Task 5 approach so far. Only using a part of the corpus since the runtimes would be too long using the whole 100 million word dataset. Aim of the task is to classify whether an adjective/adverb-noun pair in a sentence is a metaphor or not.

In [ ]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus.reader.bnc import BNCCorpusReader
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
from operator import itemgetter
import re
import pickle
from nltk import FreqDist

In [ ]:
#only using folders A-C for now
'''Casting reader output to list. For some reason,
this makes all other processing to go tens of times faster.
The downside is that the casting takes significant amount of time,
but it has to be done only once.'''
#reader = BNCCorpusReader(root='BNC/Texts/', fileids=r'[A-C]/\w*/\w*\.xml')

#Runtime with my CPU Ryzen 5 5600X ~12 minutes
#sents = list(reader.tagged_sents())
with open('sentsFull.pkl', 'rb') as input:
    sents = pickle.load(input)

In [ ]:
'''Casting reader output to list. For some reason,
this makes all other processing to go tens of times faster.
The downside is that the casting takes significant amount of time,
but it has to be done only once.'''
#Runtime with my CPU Ryzen 5 5600X ~12 minutes
#words = list(reader.words())
with open('wordsFull.pkl', 'rb') as input:
    words = pickle.load(input)

In [ ]:
#some preprocessing (removing special characters and numbers from the list of words and changing all the words into lower case)
#runtime would be about 1.5 hours if using the whole corpus, though probably (and hopefully) shorter on a better computer

wordlist = []
special_chars = ['(',')',',','"','.','!','?','-','\'','‘','’','—',':']
for w in words:
    if w not in special_chars and not w.isnumeric():
        wordlist.append(w.lower())

In [ ]:
len(wordlist)

In [ ]:
#To increase performance, we will calculate all word frequencies here at once
#Previously we calculated one at a time, resulting in hundereds of loops over wordlist

#Runtime ~15 seconds
freqDist = FreqDist(wordlist)

In [ ]:
def find_pair(sentence):
    '''
    Finds adjective/adverb-noun part-of-speech in a given sentence using nltk part-of-speech tagging. 
    Returns only the first occurence of such pair in a sentence.
    '''
    pair = []
    tagged_words = nltk.pos_tag(sentence.split())
    adjectives = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
    nouns = ['NN', 'NNS', 'NNPS', 'NNP']
    for i in range(len(tagged_words) - 1):
        word1_category = tagged_words[i][1]
        word2_category = tagged_words[i + 1][1]
        if word1_category in adjectives and word2_category in nouns:
            pair = [tagged_words[i][0].lower(), re.sub('\W+','', tagged_words[i + 1][0]).lower()]
            return pair
    return pair

def check_senses(pair):
    '''
    Given an adjective/adverb-noun pair checks that the adjective/adverb has more than one sense and the noun has an
    entry in WordNet.
    '''
    adj = pair[0]
    noun = pair[1]
    if len(wn.synsets(adj)) == 1:
        print('Adjective has only one sense!')
        return False
    elif len(wn.synsets(noun)) == 0:
        print('Noun has no entry in WordNet!')
        return False
    return True

def find_words_near(node):
    '''
    Finds nouns appearing next to a given node word by checking each sentence of the corpus individually.
    '''
    print('Looking for words appearing next to', node)
    words_near = []
    for sentence in sents:
        for i in range(len(sentence)):
            if sentence[i][0] == node:
                indexesToTry = [i - 1, i + 1]
                for index in indexesToTry:
                    if index >= 0 and index < len(sentence):
                        if sentence[index][1] and sentence[index][1] == 'SUBST':
                            words_near.append(sentence[index][0].lower())
    return words_near

def find_collocates(node, words_near):
    '''
    Finds all unique collocate nouns from a list of nouns that appear near the node word. A noun is considered a collocate
    when its mutual information to the node is greater or equal to 3. Only considers nouns that appear at least twice
    near the node word.
    '''
    print('Determining the collocates of', node)
    collocates = []
    checked = []
    #freq_node = wordlist.count(node) #Replaced by taking frequency from precalculated distribution
    freq_node = freqDist[node]
    for word in words_near:
        if word not in checked:
            checked.append(word)
            freq_near = words_near.count(word)
            if freq_near >= 2:
                #freq_collocate = wordlist.count(word) #Replaced by taking frequency from precalculated distribution
                freq_collocate = freqDist[word]
                mutual_information = calculate_mutual_information(freq_node, freq_collocate, freq_near)
                if mutual_information >= 3:
                    if (word, mutual_information) not in collocates:
                        collocates.append((word, mutual_information))
    return collocates

def calculate_mutual_information(freq_node, freq_collocate, freq_near):
    '''
    Calculates the mutual information between a node and a possible collocate using the expression (2) in
    the article https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0062343 by Neuman et al (2013).
    '''
    corpus_size = len(wordlist)
    span = 2 #maybe?
    mutual_information = np.log10((freq_near * corpus_size)/(freq_node * freq_collocate * span))/np.log10(2)
    return mutual_information

with open('WordNet2/Words.cat', 'r') as file:
    data = file.readlines()
    nouns = data[25040:94946]
    
tidy_nouns = []
for noun in nouns:
    noun = noun.replace(' (1)', '')
    noun = noun.replace('\t', '')
    noun = noun.replace('\n', '')
    noun = noun.lower()
    tidy_nouns.append(noun)

def find_classes(collocates):
    '''
    Classifies a list of nouns using WordStat. Returns a list of nouns (and their mutual information also given in
    the input list) that belong to a concrete class. The list is sorted by the mutual information value in ascending order.
    '''
    Lem = WordNetLemmatizer()

    current_class = ''
    classified_nouns = []
    
    #there should be 13 concrete classes out of the 25 classes, now there is 12 and some might be incorrect :D
    concrete_classes = ['animal', 'artifact', 'body', 'event', 'food', 'location', 'object', 'person', 'possession',
                       'plant', 'shape', 'substance']

    for target in collocates:
        if tidy_nouns.count(target[0]) > 0:
            for noun in tidy_nouns:
                #changing the noun class (e.g. when noun.food is encountered, class is 'food' until the next noun
                #class is encountered)
                if 'noun.' in noun:
                    noun_split = noun.split('.')
                    current_class = noun_split[1]
                elif noun == target[0] and current_class in concrete_classes:
                    #only adding nouns that are not already in the list (some might be in more than one conrete class)
                    if (noun, target[1]) not in classified_nouns:
                        classified_nouns.append((noun, target[1]))
    #sorting by the mutual information value
    classified_nouns = sorted(classified_nouns, key=itemgetter(1))
    return classified_nouns

def calculate_compatibility(classified_nouns, node):
    '''
    Calulates the Wu and Palmer semantic similarity
    '''
    compatible = []
    for word in classified_nouns:
        syn1 = wn.synsets(word[0])[0]
        syn2 = wn.synsets(node)[0]
        similarity = syn1.wup_similarity(syn2)
        if similarity >= 0.3:
            compatible.append(word[0])
    return compatible

In [ ]:
def is_a_metaphor(sentence):
    '''
    Determines whether a sentence includes a type III metaphor (adjective/adverb-noun metaphor) by going through a set of steps.
    '''
    pair = find_pair(sentence)
    if not pair:
        print('No pair!')
        return
    print('Pair found: ', pair)
    
    check = check_senses(pair)
    if not check:
        print('Sense check failed!')
        return
    print('Sense check ok!')
    
    adjective = pair[0]
    noun = pair[1]
    
    words_near = find_words_near(adjective)
    print('Found', len(words_near), 'nouns near', adjective)
    
    collocates = find_collocates(adjective, words_near)
    print('Found', len(collocates), 'unique collocates')
    
    if not collocates:
        print('Undetermined, not enough collocates')
        return
    
    classified_nouns = find_classes(collocates)
    print(len(classified_nouns), 'collocate words in concrete classes')
    
    if not classified_nouns:
        return 'Is a metaphor!'
    
    compatible = calculate_compatibility(classified_nouns[-3:], noun)
    print('Out of the top three collocates belonging to concrete classes,', len(compatible), 'are compatible with the noun')
    
    if compatible:
        return 'Not a metaphor!'
    return 'Is a metaphor!'

Below are a few example sentences, the counts in the first one are there just to check that the code was running okay and removed from the later test runs. Some phrases are classified correctly, some not. ~~On average it takes about 15 to 20 minutes to run is_a_metaphor() for one sentence, but again the time might be considerably shorter on better computers.~~

Due to updates each search takes only seconds.

In [ ]:
is_a_metaphor('She is such a dramatic person!')

In [ ]:
is_a_metaphor('He has a green thumb.')

In [ ]:
is_a_metaphor('I have a curious cat who likes to get into trouble.')

In [ ]:
is_a_metaphor('They stood in the dead center of the room.')

In [ ]:
is_a_metaphor('Do you have a colourful mind?')

In [ ]:
'''
Formatting of the metaphor annotated corpus: http://aclweb.org/anthology/W/W17/W17-2201.pdf
Example: destroying alexandria . sunlight is silence @4@y
@-sign separates different fields
Sentece:                        destroying alexandria . sunlight is silence
Position of the head word       4 -> sunlight
Is the expression a metaphore?  y ()
'''
annotatedSentences = []
#cleanAnnotatedSentences = [] #Not cleaning for now
headWordPosition = []
isSentenceMetaphor = []
with open("type1_metaphor_annotated.txt") as textfile:
    for line in textfile: 
        line = line.split("@")
        if(line[2] != 's'): #Exclude sentences which humans failed to classify
            annotatedSentences.append(line[0].strip())
            headWordPosition.append(line[1].strip())
            isSentenceMetaphor.append(line[2].strip())

'''
The dataset is quite problematic because the text is so unclean:
(
the night is each man 's castle . @2@y
swelling lukewarm ; her mouth is water , @5@y
& yet the earth is divinity , the sky is divinity @4@n
"i am the resurrection and the life . " @1@n
"how is the dean ? " -- "he 's just alive . " @1@n <-- especially problematic, because quotation mark is connected to the word
)
If all special characters are cleaned a way, the head word
position tag may change, and we cannot reliably verify the algorithm.
If not cleaned, the some words are left unrecognized...

For now, data is not cleaned before taking it to the algorithm
'''

In [ ]:
predictions = []
for sentence in annotatedSentences:
    if is_a_metaphor(sentence):
        predictions.append('y')
    else:
        predictions.append('n')

In [ ]:
print(len(predictions))
print(len(isSentenceMetaphor))

In [ ]:
print(predictions)

In [ ]:
print(isSentenceMetaphor)

In [ ]:
PredictedPositives = predictions.count('y')
PredictedNegatives = predictions.count('n')
TruePositives = 0
FalsePositives = 0
TrueNegatives = 0
FalseNegatives = 0
Positives = isSentenceMetaphor.count('y') #Ground truth
Negatives = isSentenceMetaphor.count('n') #Ground truth

for i in range(len(isSentenceMetaphor)):
    if predictions[i] == 'y' and isSentenceMetaphor[i] == 'y':
        TruePositives +=1
    elif predictions[i] == 'y' and isSentenceMetaphor[i] == 'n':
        FalsePositives += 1
    elif predictions[i] == 'n' and isSentenceMetaphor[i] == 'n':
        TrueNegatives += 1
    elif predictions[i] == 'n' and isSentenceMetaphor[i] == 'y':
        FalseNegatives += 1

print("Positives: " + str(Positives))
print("Negatives: " + str(Negatives))
print("Predicted positives: " + str(PredictedPositives))
print("Predicted negatives: " + str(PredictedNegatives))
print("TP: " + str(TruePositives))
print("FP: " + str(FalsePositives))
print("TN: " + str(TrueNegatives))
print("FN: " + str(FalseNegatives))



In [ ]:
accuracy = (TruePositives + TrueNegatives) / (Positives + Negatives)
print(accuracy)